In [12]:
!pip install openai pandas langchain langchain[openai] langchain-community


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import openai
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_community.llms import OpenAI

In [14]:
openai.api_key = ""

In [15]:
# JSON 파일 경로
json_file_path = "C:/SKN_3_MyProject/SKN_03_FINAL/Data/Final/processed_performace_details.json"
# JSON 파일을 데이터프레임으로 읽기
df = pd.read_json(json_file_path, lines=True)

In [16]:
df["genre"] = ""
df["story"] = ""

In [17]:
from langchain.chat_models import ChatOpenAI  # langchain에서 ChatOpenAI 가져오기

In [18]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=300,
    openai_api_key=openai.api_key,
)


In [19]:
print(openai.__version__)

1.54.5


In [20]:
total_charge = 0.0
cnt = 0

In [21]:
def get_genre_and_story(row):
    global total_charge
    global cnt
    # LangChain의 ChatPromptTemplate 사용
    prompt_template = ChatPromptTemplate.from_template("""
    다음은 뮤지컬의 정보입니다:
    제목: {title}
    상영 위치: {place}
    출연진: {cast}
    포스터 URL: {poster}
    
    1. 이 뮤지컬의 장르를 드라마/감동, 코미디/유머, 액션/스릴러, 판타지/어드벤처, 음악중심/주크박스 이 5가지 카테고리 중에 적절한 것을 골라서 적어주세요.
    2. 이 뮤지컬의 줄거리를 50~100자로 요약해서 적어주세요.
    
    아래와 같은 형식으로 답변해주세요:
    장르: <장르>
    줄거리: <줄거리>
    """)

    # 이미 genre와 story가 존재하는 경우 건너뛰기
    if row['genre'] and row['story']:
        return row['genre'], row['story']  # 기존 값을 반환
    else:
        try:
            # 프롬프트 생성
            prompt = prompt_template.format_messages(
                title=row['title'],
                place=row['place'],
                cast=row['cast'],
                poster=row['poster']
            )
            
            with get_openai_callback() as cb:  # OpenAI API 호출에 대한 콜백
                response = chat_model(prompt)  # OpenAI 모델 호출
                content = response.content
                
                total_charge += cb.total_cost
                cnt += 1
                print(f"{cnt}: 호출에 청구된 총 금액(USD): \t${total_charge}")

            genre = content.split("장르: ")[1].split("\n")[0].strip()
            story = content.split("줄거리: ")[1].strip()
            
            return genre, story
        except Exception as e:
            print(f"오류 발생: {e}")
            return "", ""

In [22]:
for idx, row in df.iterrows():
    if pd.notnull(row['genre']) and pd.notnull(row['story']):
        genre = row['genre']
        story = row['story']
    else:
        genre, story = get_genre_and_story(row)

    df.at[idx, "genre"] = genre
    df.at[idx, "story"] = story

1: 호출에 청구된 총 금액(USD): 	$0.000711
2: 호출에 청구된 총 금액(USD): 	$0.0015665000000000002
3: 호출에 청구된 총 금액(USD): 	$0.0023480000000000003


KeyboardInterrupt: 

In [12]:
print(df)

                            title  start_date    end_date  \
0                             긴긴밤  2024.10.15  2025.01.05   
1             루드윅: 베토벤 더 피아노 [정읍]  2024.10.25  2024.10.25   
2                      청공축제, 온새미로  2024.10.23  2024.10.23   
3                        유앤잇 [상주]  2024.10.26  2024.10.26   
4    보광동아트페스티벌, 해저2만리: 쓰레기섬의 아로낙스  2024.10.24  2024.10.26   
..                            ...         ...         ...   
622        날개 잃은 천사: 사람은 무엇으로 사는가  2022.07.06  2023.08.30   
623                    아기돼지야 준비해!  2022.04.23  2024.11.17   
624                    스크루테이프의 편지  2022.01.01  2024.12.28   
625                        빌의 구둣방  2021.03.20  2024.12.31   
626           빨간모자야 조심해! [청주, 세종]  2018.01.26  2025.04.06   

                        place                                 cast  \
0            링크아트센터드림 (드림 2관)  홍우진, 강정우, 이형훈, 연지현, 이정화, 설가은, 박근식 등   
1           정읍사예술회관 (정읍사예술회관)            김주호, 정재환, 김지유, 윤도영, 크리스 영   
2    민송아트홀 (구. 브로드웨이아트홀) (1관)  박다영, 구문회, 김영현, 김소진, 이예린, 박

In [13]:
df.to_csv("musicals_with_genre_and_story_ko.csv", index=False, encoding='utf-8-sig')

In [14]:
import os

# JSON 파일 경로
json_file_path = "C:/SKN_3_MyProject/SKN_03_FINAL/Data/Final/add_genre_story.json"

# 데이터프레임을 JSON 형식으로 저장
df.to_json(json_file_path, orient='records', force_ascii=False, lines=True)